In [ ]:
%autosave 0

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [1]:
import pymongo
import numpy
#import colorlover as cl

client = pymongo.MongoClient("localhost", 27017)

db = client.test # Use DB name 'test'
collection = db.TRE_Performance_Mask

In [2]:
#cursor = collection.find()
#cursor = collection.find({'Temperature_Vec_1.Mean':30}, {'Temperature_Vec_1.Mean': 1})
#cursor = collection.find({'Temperature_Vec_1.Mean':30}, {'Temperature_Vec_1.$.Mean':30})
#cursor = collection.find({}, {'Temperature_Vec_1.Mean':'true', '_id':'false'})
#cursor = collection.find({}, {'Temperature_Vec_1':'true'})

def fetch_records(cursor):
    # Find number of records
    z = cursor.count()
    print(z)
    #for record in cursor:
    #    z += 1
    #    print(z)
    #    #print(record)

    # Matrix with all TRE values. x=col, y=row, z=records
    records = numpy.array([[[0 for x in range(40)] for y in range(48)] for z in range(z)])
    # Index using [z][y][x] from now on. First index is 0
    #print(records)

    # For all TRE_Performance_Mask records
    record_index = 0
    for record in cursor:
        #print(record)
        #print(record_index)
        records[record_index] = record['TRE_Performance_Mask']
        record_index += 1

    #print(records)
    #print(records[0])
    return records

In [3]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets


def main(channel_type, rx_channel_type):
    if channel_type == 0:
        cursor = collection.find(
            {'HFF_Channel_Type':'TX'}, 
            {'TRE_Performance_Mask':'true'}) # Find all TX records and return only array
    elif channel_type == 1:
        if rx_channel_type == 0:
            cursor = collection.find(
                {'HFF_Channel_Type':'RX', 'HFF_RX_Channel_Type':'HFF_A_CHANNEL'}, 
                {'TRE_Performance_Mask':'true'})
        elif rx_channel_type == 1:
            cursor = collection.find(
                {'HFF_Channel_Type':'RX', 'HFF_RX_Channel_Type':'HFF_B_CHANNEL'}, 
                {'TRE_Performance_Mask':'true'})
        else:
            print('ERROR: Unknown RX channel type')
    else:
        print('ERROR: Unknown channel type')
        
    global records
    records = fetch_records(cursor)
    # Plot manually
    #plot_record(0)
    
    
interact(main, channel_type={'TX':0, 'RX':1}, rx_channel_type={'HFF_A_CHANNEL':0, 'HFF_B_CHANNEL':1});

2


In [9]:
####### Plotting using plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
init_notebook_mode()
    
from ipywidgets import interactive, interact
import ipywidgets as widgets
from IPython.display import display

row_label = []
for i in range(48):
    row_label.append(str(i+1))
col_label = []
for i in range(40):
    col_label.append(str(i+1))
#bupu = cl.scales['9']['seq']['BuPu']
#bupu500 = cl.interp( bupu, 500 ) # Map color scale to 500 bins
#colorscale = [[0, '#3D9970'], [150, '#001f3f']]
    
def plot_record(z):
    global records
    selected_record = records[z]
    #print(selected_record.shape)
    
    # Transpose x and y in array
    #values_trans = numpy.array(selected_record).transpose()

    # OK, ERROR, CALIBRATION_ERROR, UNKNOWN
    my_colorscale = [[0, '#33cc33'],[1, '#ff3300'],[2, '#ffff00'],[3, '#336699']]
    fig = FF.create_annotated_heatmap(
        selected_record, 
        y=row_label, 
        x=col_label,
        colorscale=my_colorscale) # 'Jet', 'Viridis'
    fig['layout'].update({
        'title':'HFF_MRM_Temperature_Status',
        'autosize':False,
        'width':'800',
        'height':'1000',
        #'margin':go.Margin(l=50, r=50, b=100, t=100, pad=4)
        #'paper_bgcolor':'#7f7f7f',
        #'plot_bgcolor':'#c7c7c7'
    })
    iplot(fig)
    
    #print('Selected index=' + str(z))
    #print(selected_values)
    
#data_selector(0)

In [5]:
interact(plot_record, z=widgets.IntSlider(min=0, max=len(records)-1, step=1, value=0));

In [8]:
from plotly.offline import download_plotlyjs, init_notebook_mode,  plot
from plotly.graph_objs import *
init_notebook_mode()

trace0 = Scatter(
    x=[1, 2, 3, 4],
    y=[10, 11, 12, 13],
    mode='markers',
    marker=dict(
        size=[40, 60, 80, 100],
    )
)
data = [trace0]
layout = Layout(
    showlegend=False,
    height=600,
    width=600,
)

fig = dict( data=data, layout=layout )

iplot(fig)  